In [ ]:
from util.util import loadAllData,get_verified_dataset
import re
import json

In [ ]:
# TODO: set the path to load your data
gp_2022 = loadAllData("/gp_2022/", True)

In [ ]:
# TODO: set the path to your result folder
# set the first argument to the base path of the result folder, second argument is th folder name, third argument a mapping file - that tells for apps in multiple sub datasets which app to include

verified = get_verified_dataset("/", "/2023_04_06/", "/verified_dataset/result.json")


In [ ]:
def sanitize_algorithm(dataset):
    result = []
    for app in dataset:
        app_algorithms = set()
        for algorithm in app:
            if '.' in algorithm or '' == algorithm or len(algorithm) < 2 or 'unknown' in algorithm or '$' in algorithm or '{}' == algorithm:
                continue
            elif 'lgorithm' in algorithm or 'UNKNOWN' in algorithm or '@' in algorithm or 'Alg' in algorithm or 'helper' in algorithm:
                continue
            elif 'aaa' == algorithm or 'value' in algorithm or 'zz' in algorithm or '_a' in algorithm or '_enve' in algorithm:
                continue
            elif "5gaj" in algorithm or 'AUTHORITY' in algorithm or "BASIC_CONS" in algorithm or '00' in algorithm or '0o' in algorithm or 'ame' in algorithm:
                continue
            app_algorithms.add(algorithm)
        result.append(app_algorithms)
    return result



In [ ]:
def cluster_algorithm(algorithm):
    algorithm = algorithm.lower()
    if algorithm.startswith('aes'):
        return 'AES'
    elif algorithm == 'arc4' or algorithm == 'arcfour' or algorithm == 'rc4':
        return 'RC4'
    elif algorithm == 'blowfish':
        return 'BLOWFISH'
    elif algorithm.startswith('cast'):
        return "CAST"
    elif algorithm.startswith('chacha'):
       # print(algorithm)
        return "ChaCha" #"ChaCha"
    elif algorithm.startswith('des'):
        if algorithm == "desede3":
            return "DES3"
        elif algorithm == "desede":
            return "DES2"
        return "DES"
    elif algorithm == "dh" or algorithm == "ecdh":
        return "DH"
    elif algorithm.startswith('grain'):
        return "GRAIN"
    elif algorithm.startswith('gost'):
        #print(algorithm)
        return "GOST"#algorithm #"GOST"
    elif algorithm.startswith('hc'):
        return "HC"
    elif algorithm.startswith('hmac'):
        return "HMAC"
    elif algorithm.startswith('md5'):
        return "MD5"
    elif algorithm.startswith('noekeon'):
        return "Noekeon"
    elif algorithm.startswith('pbe'):
        return "PBE"
    elif algorithm.startswith('pbkdf'):
        return "PBKDF"
    elif algorithm.startswith('poly'):
        #print(algorithm)

        return "ChaCha" #"Poly"
    elif algorithm == 'rc2':
        return "RC2"
    elif algorithm == 'rc5':
        return "RC5"
    elif algorithm == 'rc6':
        return "RC6"
    elif algorithm.startswith('rsa'):
        return "RSA"
    elif algorithm.startswith('hkdf'):
        return "HKDF"
    elif algorithm.startswith('zuc'):
        return "ZUC"
    elif algorithm.startswith('twofish'):
        return "Twofish"
    elif algorithm.startswith('tnepres'):
        return "Tnepres"
    elif algorithm.startswith('threefish'):
        return "Threefish"
    elif algorithm.startswith('skipjack'):
        return "Skipjack"
    elif algorithm.startswith('skein'):
        return "Skein"
    elif algorithm.startswith('shacal'):
        return "Shacal"
    elif algorithm.startswith('serpent'):
        return "Serpent"
    elif algorithm.startswith('salsa'):
        return "Salsa"
    elif algorithm.startswith('sha1'):
        return "SHA1"
    elif algorithm.startswith('sha'):
        return "SHA"
    elif algorithm.startswith('scrypt'):
        return "SCRYPT"
    elif algorithm.startswith('rijndael'):
        return "Rijndael"
    else:
        return None

In [ ]:
def get_all_values_from_valueset(value_sets, id):
    result = set()
    for vs in value_sets:
        data = vs.get(str(id), [])
        for d in data:
            result.add(d)
    return result


def add_values(valuePoint):
    algos = set()
    for v in valuePoint:
        ca = cluster_algorithm(v)
        if  ca != None:
            algos.add(ca)

    return len(algos) <5 # Some value point add all different algorithms, likely fp (general methods not specifically used for encryption) so exclude them





def extract_algorithm_from_json(app):
    algorithm = set()
    jsonData = app.crypto
    for vp in jsonData.get("ValuePoints", []):
        signature = vp.get("appendix", {}).get("sigatureInApp", "")
        if signature != None and signature != "":
            if "javax.crypto.spec.SecretKeySpec: void <init>(byte[],int,int,java.lang.String)" in signature:
                values = get_all_values_from_valueset(vp.get("ValueSet", []), 3)
                if add_values(values):
                    algorithm.update(values)

            elif "javax.crypto.spec.SecretKeySpec: void <init>(byte[],java.lang.String)" in signature:
                values = get_all_values_from_valueset(vp.get("ValueSet", []), 1)
                if add_values(values):
                    algorithm.update(values)
            elif "javax.security.auth.kerberos.KerberosKey: void <init>(javax.security.auth.kerberos.KerberosPrincipal,char[],java.lang.String)" in signature:
                values = get_all_values_from_valueset(vp.get("ValueSet", []), 2)
                if add_values(values):
                    algorithm.update(values)
            elif "javax.crypto.spec.DESedeKeySpec:" in signature:
                algorithm.add("DES")
            elif "javax.crypto.spec.DESKeySpec:" in signature:
                algorithm.add("DES")
            elif "javax.crypto.spec.DHPrivateKeySpec:" in signature:
                algorithm.add("DH")
            elif "javax.crypto.spec.DSAPrivateKeySpec:" in signature:
                algorithm.add("DSA")
            elif "javax.crypto.spec.ECPrivateKeySpec:" in signature:
                algorithm.add("EC")
            elif "javax.crypto.spec.PBEKeySpec:" in signature:
                continue
            elif "javax.crypto.spec.PKCS8EncodedKeySpec:" in signature:
                continue
            elif "javax.crypto.spec.RSAPrivateKeySpec:" in signature:
                algorithm.add("RSA")
            elif "javax.crypto.spec.X509EncodedKeySpec:" in signature:
                continue
            elif "javax.crypto.spec.RSAMultiPrimePrivateCrtKeySpec:" in signature:
                algorithm.add("RSA")
            elif "javax.crypto.spec.EncodedKeySpec:" in signature:
                continue
    return algorithm


def algorithms_used(dataset):
    result = []
    for app in dataset:
        result.append(extract_algorithm_from_json(app))
    return result

In [ ]:
gp_2022_algo = algorithms_used(gp_2022)
iot_ver_algo = algorithms_used(verified)

In [ ]:
def get_unique_algorithm(dataset):
    result = set()
    for app in dataset:
        for alg in app:
            result.add(alg)

    return result

In [ ]:
get_unique_algorithm(iot_ver_algo)

In [ ]:
sanitized_iot_ver = sanitize_algorithm(iot_ver_algo)


In [ ]:
sanitized_iot_ver

In [ ]:

sanitized_gp_2022 = sanitize_algorithm(gp_2022_algo)

In [ ]:
def is_unsafe(algorithm):
    algorithm = algorithm.lower()
    return "gost" in algorithm or 'rc' in algorithm or 'arcfour' in algorithm or 'des' in algorithm or 'gost' in algorithm or 'shacal' in algorithm or 'skipjack' in algorithm

def classify_algorithms(dataset):
    count = 0
    for app in dataset:
        for algorithm in app:
            if is_unsafe(algorithm):
                count = count + 1
                break
    return count





In [ ]:
def is_encryption_in_table(algorithm):
    if algorithm is None:
        return False
    algorithm = algorithm.lower()
    return algorithm == "aes" or algorithm == "chacha" or algorithm == "dh" or algorithm == "rsa" or algorithm == "blowfish" or algorithm == "serpent" or algorithm == "des" or algorithm == "3des" or algorithm == "ghost" or algorithm == "rc4"

In [ ]:
def number_of_apps_with_encryption(dataset):
    result = 0
    for app in dataset:
        has_encryption = False
        for x in app:
            if is_encryption_in_table(cluster_algorithm(x)):
                has_encryption = True

        if has_encryption:
            result = result + 1
    return result



In [ ]:
def is_encryption(algorithm):
    algorithm = algorithm.lower()
    return algorithm == "aes" or algorithm == "rc4" or algorithm == "blowfish" or algorithm == "cast" or algorithm == "chacha" or algorithm == "des" or algorithm == "des" or algorithm == "dh" or algorithm == "grain" or algorithm == "ghost" or algorithm == "hc" or algorithm == "noekeon" or algorithm.startswith('rc') or algortihm == 'rsa' or algorithm == "zuc" or algorithm=="twofish" or algorithm == 'threefish' or algorithm == "tnepres" or algorithm == "skipjack" or algorithm == "shacal" or algorithm == "serpent" or algorithm =="salsa" or algorithm == "scrypt" or algorithm == "rijndael"

In [ ]:
def cluster_dataset(dataset):
    result = {}
    for app in dataset:
        all_algorithm_apps = set()
        for x in app:
            all_algorithm_apps.add(cluster_algorithm(x))

        for algo in all_algorithm_apps:
            result[algo] = result.get(algo, 0) + 1
    return result


In [ ]:
clusterd_iot_ver = cluster_dataset(sanitized_iot_ver)

In [ ]:
clusterd_gp_2022 = cluster_dataset(sanitized_gp_2022)


In [ ]:
clusterd_iot_ver


In [ ]:
number_of_apps_with_encryption(sanitized_iot_ver)

In [ ]:
keys = set()
keys = keys.union(set(clusterd_gp_2022.keys()))
keys = keys.union(set(clusterd_iot_ver.keys()))

In [ ]:
gp_2022_column = []
iot_ver_column = []

algorithms_column = []
for algo in keys:
    if algo == None:
        continue
    algorithms_column.append(algo)
    gp_2022_column.append(clusterd_gp_2022.get(algo, 0))
    iot_ver_column.append(clusterd_iot_ver.get(algo, 0))


In [ ]:
import pandas as pd

data = {"algorithm":  algorithms_column,
        'GP-2022': gp_2022_column,
        'IoT-VER': iot_ver_column,

        }

df = pd.DataFrame(data)


In [ ]:
df # remove algos where >x algos per value point

In [ ]:
def format_table(table):
    gp_2022_column = []
    iot_ver_column = []

    alogrithm_column = []
    for row in table.iterrows():
        alogrithm_column.append(row[1]["algorithm"])
        gp_2022_column.append("{} ({:.2f}%)".format(row[1]['GP-2022'],row[1]['GP-2022']/947*100))

        iot_ver_column.append("{} ({:.2f}%)".format(row[1]['IoT-VER'],row[1]['IoT-VER']/9889 *100))

    return pd.DataFrame({"algorithm": alogrithm_column, "GP 2022": gp_2022_column, "IoT VER": iot_ver_column})


In [ ]:
formattedDF = format_table(df)
formattedDF = formattedDF.sort_values(by=['algorithm'])

In [ ]:
formattedDF

In [ ]:
print(formattedDF.to_latex(index=False))

In [ ]:
"{:.2f}%".format(number_of_apps_with_encryption(sanitized_gp_2022)/947 * 100)

In [ ]:
number_of_apps_with_encryption(sanitized_gp_2022)

In [ ]:
classify_algorithms(sanitized_gp_2022)

In [ ]:
classify_algorithms(sanitized_iot_ver)

In [ ]:
"{:.2f}%".format(classify_algorithms(sanitized_gp_2022)/number_of_apps_with_encryption(sanitized_gp_2022) * 100)

In [ ]:
"{:.2f}%".format(classify_algorithms(sanitized_iot_ver)/number_of_apps_with_encryption(sanitized_iot_ver) * 100)

In [ ]:
"{:.2f}%".format(number_of_apps_with_encryption(sanitized_iot_ver)/9889 * 100 )

In [ ]:
number_of_apps_with_encryption(sanitized_iot_ver)

In [ ]:
"{:.2f}%".format(classify_algorithms(sanitized_gp_2022)/947 * 100)

In [ ]:
"{:.2f}%".format(classify_algorithms(sanitized_iot_ver)/9889 * 100 )

In [ ]:
empty_array = re.compile("\[(0,\s?)*0?\]")
# 2 or more .
def getHardcodedInformation(values):
    result = set()
    for v in values:
        lower_v = v.lower()
        array_replace = lower_v.replace("[", "")
        array_replace = array_replace.replace("]", "")
        array_replace = array_replace.replace(" ", "")
        array_replace = array_replace.replace(",", "")

        if lower_v.isnumeric() and int(lower_v) != 0 and int(lower_v) != -1:
            result.add(v)
            continue
        elif lower_v.isnumeric():
            continue


        if empty_array.match(lower_v) or len(v) < 6 or 'key' in lower_v or 'secret' in lower_v or "unknown" in lower_v or v.count(".")>1 or "sharedpreferences" in lower_v or "@" in lower_v:
            continue
        elif "fromui" in lower_v or 'intent' in lower_v or 'password' in lower_v or "sqlite" in lower_v or "ascii" in lower_v:
            continue
        elif v[0] != '[' and v.count(' ')> 1: #[2, 4, 6, 6] is fine but not asdf asdf adf
            continue
        elif len(array_replace) < 6 or 'key' in array_replace or 'secret' in array_replace or "unknown" in array_replace or array_replace.count(".")>1 or "sharedpreferences" in array_replace or "@" in array_replace:
            continue
        elif "fromui" in array_replace or 'intent' in array_replace or 'password' in array_replace or "sqlite" in array_replace or "context" in array_replace:
            continue
        elif "encry" in lower_v or "help" in lower_v or "android" in lower_v or "user" in lower_v or "json" in lower_v or ("(" in lower_v and ")" in lower_v) or "salt" in lower_v or "provider" in lower_v:
            continue
        elif "crypt" in lower_v or "cordova" in lower_v or "cordova" in array_replace or "emulator" in array_replace or "emulator" in lower_v or "undefined" in lower_v or "object" in lower_v:
            continue
        elif "serial" in lower_v or "engine" in lower_v or "serial" in array_replace or "engine" in array_replace or "manager" in lower_v or "authent" in lower_v or "calc" in lower_v or "login" in lower_v or "config" in lower_v or "mail" in lower_v:
            continue
        elif "prng" in lower_v or "sha" in lower_v or "fragment" in lower_v or "aes" in lower_v or "rsa" in lower_v or "des" in lower_v or "pkcs" in lower_v:
            continue
        elif "store" in array_replace or 'encrypt' in array_replace or 'integrity' in array_replace or "kotlin" in array_replace or "kotlin" in lower_v:
            continue
        elif lower_v.count("0") >3 or (lower_v[0] == "[" and lower_v[len(lower_v) - 1] == "]" and lower_v.count(",")<5):
            continue
        else:
            result.add(v)
    return list(result)



def extract_keys_from_json(app):
    all_data = set()
    #extracted_information = {"type": ["key", "iv"], "key": ["asdf"], ...}
    jsonData = app.crypto
    for vp in jsonData.get("ValuePoints", []):
        signature = vp.get("appendix", {}).get("sigatureInApp", "")
        if signature != None and signature != "":
            if "javax.security.auth.kerberos.KerberosKey: void <init>(javax.security.auth.kerberos.KerberosPrincipal,byte[],int,int)" in signature or "KerberosKey: void <init>(javax.security.auth.kerberos.KerberosPrincipal,char[],java.lang.String)" in signature:
                #1
                values = get_all_values_from_valueset(vp.get("ValueSet", []), 1)
                potentialHardcoded = getHardcodedInformation(values)
                if len(potentialHardcoded) != 0:
                    extracted_information = {"type": "key", "key": potentialHardcoded}
                    all_data.add(json.dumps(extracted_information))
            elif "javax.crypto.spec.SecretKeySpec: void <init>(byte[],int,int,java.lang.String)" in signature:
                values_iv = get_all_values_from_valueset(vp.get("ValueSet", []), 0)
                values_offset = get_all_values_from_valueset(vp.get("ValueSet", []), 1)
                values_len = get_all_values_from_valueset(vp.get("ValueSet", []), 2)

                extracted_information ={"type": "iv"}
                potential_hardcoded_iv = getHardcodedInformation(values_iv)
                potential_hardcoded_offset = getHardcodedInformation(values_offset)
                potential_hardcoded_len = getHardcodedInformation(values_len)

                if len(potential_hardcoded_iv) != 0:
                    extracted_information = { "iv": potential_hardcoded_iv}
                if len(potential_hardcoded_offset) != 0:
                    extracted_information["offset"] = potential_hardcoded_offset
                if len(potential_hardcoded_len) != 0:
                    extracted_information["len"] = potential_hardcoded_len

                all_data.add(json.dumps(extracted_information))

            elif "javax.crypto.spec.SecretKeySpec: void <init>(byte[],java.lang.String)" in signature:
                values = get_all_values_from_valueset(vp.get("ValueSet", []), 0)
                potentialHardcoded = getHardcodedInformation(values)
                if len(potentialHardcoded) != 0:
                    extracted_information = {"type": "key", "key": potentialHardcoded}
                    all_data.add(json.dumps(extracted_information))
            elif "javax.crypto.spec.IvParameterSpec: void <init>(byte[],int,int)" in signature:
                values_iv = get_all_values_from_valueset(vp.get("ValueSet", []), 0)
                values_offset = get_all_values_from_valueset(vp.get("ValueSet", []), 1)
                values_len = get_all_values_from_valueset(vp.get("ValueSet", []), 2)

                extracted_information ={"type": "iv"}
                potential_hardcoded_iv = getHardcodedInformation(values_iv)
                potential_hardcoded_offset = getHardcodedInformation(values_offset)
                potential_hardcoded_len = getHardcodedInformation(values_len)

                if len(potential_hardcoded_iv) != 0:
                    extracted_information = { "iv": potential_hardcoded_iv}
                if len(potential_hardcoded_offset) != 0:
                    extracted_information["offset"] = potential_hardcoded_offset
                if len(potential_hardcoded_len) != 0:
                    extracted_information["len"] = potential_hardcoded_len

                all_data.add(json.dumps(extracted_information))

            elif "javax.crypto.spec.DESedeKeySpec: void <init>(byte[])" in signature:
                values = get_all_values_from_valueset(vp.get("ValueSet", []), 0)
                potentialHardcoded = getHardcodedInformation(values)
                if len(potentialHardcoded) != 0:
                    extracted_information = {"type": "key", "key": potentialHardcoded}
                    all_data.add(json.dumps(extracted_information))
            elif "javax.crypto.spec.DESedeKeySpec: void <init>(byte[],int)" in signature:
                values_key = get_all_values_from_valueset(vp.get("ValueSet", []), 0)
                values_offset = get_all_values_from_valueset(vp.get("ValueSet", []), 1)
                extracted_information ={"type": "key"}
                potential_hardcoded_key = getHardcodedInformation(values_key)
                potential_hardcoded_offset = getHardcodedInformation(values_offset)
                if len(potential_hardcoded_key) != 0:
                    extracted_information = { "key": potential_hardcoded_key}
                if len(potential_hardcoded_offset) != 0:
                    extracted_information["offset"] = potential_hardcoded_offset

                if extracted_information != {"type": "key"}:
                    all_data.add(json.dumps(extracted_information))

            elif "javax.crypto.spec.DHPrivateKeySpec: void <init>(java.math.BigInteger,java.math.BigInteger,java.math.BigInteger)" in signature:
                values = get_all_values_from_valueset(vp.get("ValueSet", []), 0)
                values = get_all_values_from_valueset(vp.get("ValueSet", []), 0)
                values = get_all_values_from_valueset(vp.get("ValueSet", []), 0)
                # TODO

            elif "javax.crypto.spec.DSAPrivateKeySpec: void <init>(java.math.BigInteger,java.math.BigInteger,java.math.BigInteger,java.math.BigInteger)" in signature:
                values = get_all_values_from_valueset(vp.get("ValueSet", []), 0)
                values = get_all_values_from_valueset(vp.get("ValueSet", []), 0)
                values = get_all_values_from_valueset(vp.get("ValueSet", []), 0)
                values = get_all_values_from_valueset(vp.get("ValueSet", []), 0)
                # TODO

            elif "javax.crypto.spec.ECPrivateKeySpec: void <init>(java.math.BigInteger,java.security.spec.ECParameterSpec)" in signature:
                values = get_all_values_from_valueset(vp.get("ValueSet", []), 0)
                potentialHardcoded = getHardcodedInformation(values)
                if len(potentialHardcoded) != 0:
                    extracted_information = {"type": "key", "key": potentialHardcoded}
                    all_data.append(json.dumps(extracted_information))
            elif "javax.crypto.spec.PBEKeySpec: void <init>(char[])" in signature:
                values = get_all_values_from_valueset(vp.get("ValueSet", []), 0)
                potentialHardcoded = getHardcodedInformation(values)
                if len(potentialHardcoded) != 0:
                    extracted_information = {"type": "key", "key": potentialHardcoded}
                    all_data.add(json.dumps(extracted_information))
            elif "javax.crypto.spec.PBEKeySpec: void <init>(char[],byte[],int)" in signature:
                values_key = get_all_values_from_valueset(vp.get("ValueSet", []), 0)
                values_offset = get_all_values_from_valueset(vp.get("ValueSet", []), 1)
                extracted_information ={"type": "key"}
                potential_hardcoded_key = getHardcodedInformation(values_key)
                potential_hardcoded_offset = getHardcodedInformation(values_offset)
                if len(potential_hardcoded_key) != 0:
                    extracted_information = { "key": potential_hardcoded_key}
                if len(potential_hardcoded_offset) != 0:
                    extracted_information["offset"] = potential_hardcoded_offset

                if extracted_information != {"type": "key"}:
                    all_data.add(json.dumps(extracted_information))

            elif "javax.crypto.spec.PBEKeySpec: void <init>(char[],byte[],int,int)" in signature:
                #char[] password, byte[] salt, int iterationCount, int keyLength

                values_password = get_all_values_from_valueset(vp.get("ValueSet", []), 0)
                values_salt = get_all_values_from_valueset(vp.get("ValueSet", []), 1)
                values_iteration_count = get_all_values_from_valueset(vp.get("ValueSet", []), 2)
                values_key_length = get_all_values_from_valueset(vp.get("ValueSet", []), 3)

                extracted_information ={"type": "key"}
                potential_hardcoded_password = getHardcodedInformation(values_password)
                potential_hardcoded_salt = getHardcodedInformation(values_salt)
                potential_hardcoded_iteration = getHardcodedInformation(values_iteration_count)
                potential_hardcoded_length = getHardcodedInformation(values_key_length)

                if len(potential_hardcoded_password) != 0:
                    extracted_information = { "key": potential_hardcoded_password}
                extracted_information["salt"] = potential_hardcoded_salt
                if len(potential_hardcoded_iteration) != 0:
                    extracted_information["iteration"] = potential_hardcoded_iteration
                if len(potential_hardcoded_length) != 0:
                    extracted_information["iteration"] = potential_hardcoded_length

                if extracted_information != {"type": "key"}:
                    all_data.add(json.dumps(extracted_information))
            elif "javax.crypto.spec.PKCS8EncodedKeySpec: void <init>(byte[])" in signature:
                values = get_all_values_from_valueset(vp.get("ValueSet", []), 0)
                potentialHardcoded = getHardcodedInformation(values)
                if len(potentialHardcoded) != 0:
                    extracted_information = {"type": "key", "key": potentialHardcoded}
                    all_data.add(json.dumps(extracted_information))
            elif "void <init>(java.math.BigInteger,java.math.BigInteger)" in signature:
                values = get_all_values_from_valueset(vp.get("ValueSet", []), 0)
                values = get_all_values_from_valueset(vp.get("ValueSet", []), 1)

                #TODO:

            elif "javax.crypto.spec.X509EncodedKeySpec: void <init>(byte[])" in signature:
                values = get_all_values_from_valueset(vp.get("ValueSet", []), 0)
                potentialHardcoded = getHardcodedInformation(values)
                if len(potentialHardcoded) != 0:
                    extracted_information = {"type": "key", "key": potentialHardcoded}
                    all_data.append(json.dumps(extracted_information))
            elif "javax.crypto.spec.RSAMultiPrimePrivateCrtKeySpec: void <init>(java.math.BigInteger,java.math.BigInteger,java.math.BigInteger,java.math.BigInteger,java.math.BigInteger,java.math.BigInteger,java.math.BigInteger,java.math.BigInteger,java.security.spec.RSAOtherPrimeInfo[])" in signature:
                values = get_all_values_from_valueset(vp.get("ValueSet", []), 0)
                values = get_all_values_from_valueset(vp.get("ValueSet", []), 1)
                values = get_all_values_from_valueset(vp.get("ValueSet", []), 2)
                values = get_all_values_from_valueset(vp.get("ValueSet", []), 3)
                values = get_all_values_from_valueset(vp.get("ValueSet", []), 4)
                values = get_all_values_from_valueset(vp.get("ValueSet", []), 5)
                values = get_all_values_from_valueset(vp.get("ValueSet", []), 6)
                values = get_all_values_from_valueset(vp.get("ValueSet", []), 7)
                values = get_all_values_from_valueset(vp.get("ValueSet", []), 8)

                # TODO:

            elif "javax.crypto.spec.RSAPrivateCrtKeySpec: void <init>(java.math.BigInteger,java.math.BigInteger,java.math.BigInteger,java.math.BigInteger,java.math.BigInteger,java.math.BigInteger,java.math.BigInteger,java.math.BigInteger)" in signature:
                values = get_all_values_from_valueset(vp.get("ValueSet", []), 0)
                values = get_all_values_from_valueset(vp.get("ValueSet", []), 1)
                values = get_all_values_from_valueset(vp.get("ValueSet", []), 2)
                values = get_all_values_from_valueset(vp.get("ValueSet", []), 3)
                values = get_all_values_from_valueset(vp.get("ValueSet", []), 4)
                values = get_all_values_from_valueset(vp.get("ValueSet", []), 5)
                values = get_all_values_from_valueset(vp.get("ValueSet", []), 6)
                values = get_all_values_from_valueset(vp.get("ValueSet", []), 7)

                # TODO:

            elif "javax.crypto.spec.EncodedKeySpec: void <init>(byte[])" in signature:
                values = get_all_values_from_valueset(vp.get("ValueSet", []), 0)
                potentialHardcoded = getHardcodedInformation(values)
                if len(potentialHardcoded) != 0:
                    extracted_information = {"type": "key", "key": potentialHardcoded}
                    all_data.add(json.dumps(extracted_information))


    result = []
    for a in all_data:
        result.append(json.loads(a))
    return result


def extract_keys(dataset):
    result = []
    for app in dataset:
        result.append(extract_keys_from_json(app))
    return result

In [ ]:

extracted_keys_gp_2022 = extract_keys(gp_2022)
extracted_keys_iot_ver = extract_keys(verified)

In [ ]:
# parse numbers -> if >0 fine
#check parameters somethimes things in results like PBEWithSHA256And256BitAES NativePRNG RSA/ECB/PKCS1Padding

In [ ]:
def is_hardcoded(app):
    has_iv = False
    iv  = None
    key_data_available = False
    for data in app:
        if data.get("type", None) == "iv":
            has_iv = True
            iv = data.get("iv", None)
        elif data.get("type", None) == "key" and len(data.get("key", [])) > 0:
            if data.get("salt", None) != None:
                if len(data.get("salt", [])) == 0:
                    continue

            key_data_available = True

    return has_iv == True and key_data_available == True or has_iv == False and key_data_available


def analyze_extracted_keys(dataset):
    count = 0
    for app in dataset:
        if is_hardcoded(app):
            count = count + 1
    return count

In [ ]:
analyze_extracted_keys(extracted_keys_gp_2022)

In [ ]:
analyze_extracted_keys(extracted_keys_iot_ver)

In [ ]:
analyze_extracted_keys(extracted_keys_gp_2022)/number_of_apps_with_encryption(sanitized_gp_2022) * 100

In [ ]:
analyze_extracted_keys(extracted_keys_iot_ver)/number_of_apps_with_encryption(sanitized_iot_ver) * 100

In [ ]:
new_row = ["hardcoded crypto material",analyze_extracted_keys(extracted_keys_2018), analyze_extracted_keys(extracted_keys_2021), analyze_extracted_keys(extracted_keys_2022), analyze_extracted_keys(extracted_keys_gp_2022), analyze_extracted_keys(extracted_keys_iot_ver) ]

In [ ]:
df.loc[len(df)] = new_row

In [ ]:
formattedDF = format_table(df)
formattedDF = formattedDF.sort_values(by=['algorithm'])

In [ ]:
print(formattedDF.to_latex(index=False))